<a href="https://colab.research.google.com/github/gwcwang/gwcwang.github.io/blob/master/CerealPredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cereal attributes and ratings. EDA & rating prediction. Rating predictions based on code from 
https://www.kaggle.com/madhurisivalenka/basic-machine-learning-with-red-wine-quality-data. Raw data from 80 Cereals at https://www.kaggle.com/crawford/80-cereals

Load packages and file. Convert file to dataframe. Convert sugar measurement into category data. 

In [0]:
from google.colab import files
uploaded = files.upload()


Saving cereal.csv to cereal.csv


In [0]:
import pandas as pd 
import datetime
!pip install -U ggplot


Requirement already up-to-date: ggplot in /usr/local/lib/python3.6/dist-packages (0.11.5)


In [0]:
from ggplot import *
crl = pd.read_csv("cereal.csv") 




ParserError: ignored

In [0]:
sugars = []
for i in crl['sugars']:
    if i <= 5:
        sugars.append('lo')
    elif i >= 10:
        sugars.append('hi')
    else :
        sugars.append('mid')
  
crl['SugarHiLo'] = sugars

crl.head()


Use ggplot to make a facet plot cereal data. Calories vs. rating by cereal company. Additional attributes with size for carbs, color for sugar content, and shape for type (hot/cold)

In [0]:
ggplot(aes(x = "calories", y = "rating", colour = "SugarHiLo", shape = "type"),data = crl) + geom_jitter() + geom_point(aes(size = "carbo")) + facet_wrap("mfr")


Correlation heatmap using seaborn

In [0]:
import seaborn 
import matplotlib.pyplot as plt
corr_crl = crl.corr()

plt.subplots(figsize=(20,15))
ax = plt.axes()
ax.set_title("Correlation of different cereal attribute")

seaborn.heatmap(corr_crl, 
            xticklabels=corr_crl.columns.values,
            yticklabels=corr_crl.columns.values)


Change type into binary values, bucket rating into 3 review categories (1,2,3)

In [0]:
crl['Type_binary']=crl['type'].map({'C':1,'H':0})
crl.head()

In [0]:
reviews = []
for i in crl['rating']:
    if i >= 0 and i <= 33:
        reviews.append('1')
    elif i > 33 and i <= 66:
        reviews.append('2')
    elif i > 66 and i <= 100:
        reviews.append('3')
crl['Reviews'] = reviews

x = crl.drop(['name','mfr','type', 'SugarHiLo'],axis=1)
y = crl['Reviews']



Principal component analysis to reduce collinearity

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
print(x)

In [0]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA()
x_pca = pca.fit_transform(x)
#plot the graph to find the principal components
plt.figure(figsize=(10,10))
plt.plot(np.cumsum(pca.explained_variance_ratio_), 'ro-')
plt.grid()

90% of variation from 7 x-value components. Create train and test data


In [0]:
pca_new = PCA(n_components=7)
x_new = pca_new.fit_transform(x)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
x_train, x_test, y_train, y_test = train_test_split(x_new, y, test_size = 0.5)




Logistic Regression with confusion matrix and accurary

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_predict = lr.predict(x_test)

lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print(lr_conf_matrix)
print(lr_acc_score*100)

Decision tree with confusion matrix and accurary

In [0]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
dt_predict = dt.predict(x_test)

dt_conf_matrix = confusion_matrix(y_test, dt_predict)
dt_acc_score = accuracy_score(y_test, dt_predict)
print(dt_conf_matrix)
print(dt_acc_score*100)

NaiveBayes with confusion matrix and accurary

In [0]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
nb_predict=nb.predict(x_test)

nb_conf_matrix = confusion_matrix(y_test, nb_predict)
nb_acc_score = accuracy_score(y_test, nb_predict)
print(nb_conf_matrix)
print(nb_acc_score*100)

Random forest with confusion matrix and accurary

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_predict=rf.predict(x_test)

rf_conf_matrix = confusion_matrix(y_test, rf_predict)
rf_acc_score = accuracy_score(y_test, rf_predict)
print(rf_conf_matrix)
print(rf_acc_score*100)